<a href="https://colab.research.google.com/github/ranwiththecode/high-fantasy-data-analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

!gcloud config set project stable-course-461105-k9  # Replace with your actual project ID
!gcloud auth list  # Verify your active account

Updated property [core/project].
     Credentialed Accounts
ACTIVE  ACCOUNT
*       ranjani.y.iyer@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [ ]:
from google.colab import auth
auth.authenticate_user()

# Set project and grant permissions
!gcloud config set project stable-course-461105-k9  # REPLACE with your actual project ID
!gcloud projects add-iam-policy-binding stable-course-461105-k9 \
    --member=user:ranjani.y.iyer@gmail.com \
    --role=roles/bigquery.user

Updated property [core/project].
Updated IAM policy for project [stable-course-461105-k9].
bindings:
- members:
  - serviceAccount:service-807385999782@gcp-sa-vertex-nb.iam.gserviceaccount.com
  role: roles/aiplatform.colabServiceAgent
- members:
  - serviceAccount:service-807385999782@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-807385999782@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - user:ranjani.y.iyer@gmail.com
  role: roles/bigquery.user
- members:
  - serviceAccount:807385999782@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-807385999782@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-807385999782@gcf-admin-robot.iam.gserviceaccount.com
  role: roles/cloudfunctions.serviceAgent
- members:
  - serviceAccount:service-80

In [ ]:
# First, run this authentication cell separately before your main code
from google.colab import auth
auth.authenticate_user()

# Then run your BigQuery code

In [ ]:
import pandas as pd
from google.cloud import bigquery
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from tqdm import tqdm

# Initialize NLTK
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Configuration (UPDATE THESE)
CONFIG = {
    "project_id": "stable-course-461105-k9",      # Case-sensitive
    "dataset_id": "google_imports",   # Your dataset
    "table_id": "pubs_2000_2005",           # Your table
    "drive_path": "/content/drive/MyDrive/Goodreads_Data/",
    "input_file": "eragon_paolini_clean.csv"  # Your cleaned file
}

def analyze_sentiment(text):
    """Calculate sentiment score using VADER"""
    if not isinstance(text, str) or not text.strip():
        return None
    return sid.polarity_scores(text)['compound']

def process_and_update():
    """End-to-end processing pipeline"""
    # Initialize BigQuery client
    bq_client = bigquery.Client(project=CONFIG['project_id'])

    # 1. Load cleaned data
    input_path = os.path.join(CONFIG['drive_path'], CONFIG['input_file'])
    print(f"\nLoading data from: {input_path}")
    df = pd.read_csv(input_path)

    # 2. Calculate sentiment (with progress bar)
    print("Analyzing sentiment...")
    tqdm.pandas(desc="Processing reviews")
    df['sentiment'] = df['clean_text'].progress_apply(analyze_sentiment)

    # 3. Prepare for BigQuery update
    table_ref = f"{CONFIG['project_id']}.{CONFIG['dataset_id']}.{CONFIG['table_id']}"
    update_df = df[['id', 'sentiment']]  # Columns to update

    # 4. Execute BigQuery update
    try:
        print(f"\nUpdating BigQuery table: {table_ref}")
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE",
            create_disposition="CREATE_IF_NEEDED"
        )

        job = bq_client.load_table_from_dataframe(
            update_df,
            table_ref,
            job_config=job_config
        )
        job.result()  # Wait for completion

        print(f"✅ Success! Updated {len(update_df)} rows")
        print(f"Sample updated data:\n{update_df.head(3)}")

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        if hasattr(e, 'errors'):
            for error in e.errors:
                print(f"Detail: {error['message']}")

# Run the pipeline
if __name__ == "__main__":
    process_and_update()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!



Loading data from: /content/drive/MyDrive/Goodreads_Data/eragon_paolini_clean.csv
Analyzing sentiment...


Processing reviews: 100%|██████████| 24722/24722 [00:23<00:00, 1034.54it/s]



Updating BigQuery table: stable-course-461105-k9.google_imports.pubs_2000_2005
✅ Success! Updated 24722 rows
Sample updated data:
                                                  id  sentiment
0  kca://review:goodreads/amzn1.gr.review:goodrea...    -0.9804
1  kca://review:goodreads/amzn1.gr.review:goodrea...     0.9729
2  kca://review:goodreads/amzn1.gr.review:goodrea...    -0.6458
